In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

def preprocess_six_congress_data():
    """
    Preprocess the 6-congress dataset (113th-118th)
    """
    
    print("="*60)
    print("PREPROCESSING 6-CONGRESS DATASET (113th-118th)")
    print("="*60)
    
    # Load the full dataset
    df = pd.read_csv('../data/bills_with_features_full.csv')
    print(f"Loaded {len(df)} bills from 6 congresses")
    
    # Check which congresses we have
    print("\nCongresses in dataset:")
    print(df['congress'].value_counts().sort_index())
    
    return df

def calculate_smart_days_active(df):
    """
    Calculate days active using the latest action date instead of current date
    This prevents the model from using time as a crutch
    """
    
    print("\n" + "="*60)
    print("CALCULATING SMART DAYS ACTIVE")
    print("="*60)
    
    # Convert dates
    df['introduced_date'] = pd.to_datetime(df['introduced_date'], errors='coerce')
    df['latest_action_date'] = pd.to_datetime(df['latest_action_date'], errors='coerce')
    
    # Calculate days between introduction and latest action
    df['days_to_latest_action'] = (df['latest_action_date'] - df['introduced_date']).dt.days
    
    # For bills with invalid dates, use a default
    df['days_to_latest_action'] = df['days_to_latest_action'].fillna(30)
    
    # Clip to reasonable range (1 day to 2 years)
    df['days_active'] = df['days_to_latest_action'].clip(1, 730)
    
    # Also create a "legislative_velocity" feature
    df['legislative_velocity'] = df['action_count'] / (df['days_active'] + 1)
    
    print(f"Days active statistics:")
    print(f"- Mean: {df['days_active'].mean():.1f} days")
    print(f"- Median: {df['days_active'].median():.1f} days")
    print(f"- Min: {df['days_active'].min()}")
    print(f"- Max: {df['days_active'].max()}")
    
    # Remove the old days_since_introduction to avoid confusion
    if 'days_since_introduction' in df.columns:
        df = df.drop('days_since_introduction', axis=1)
    
    return df

def identify_outcomes_by_congress(df):
    """
    Identify bill outcomes considering which congresses have ended
    """
    
    print("\n" + "="*60)
    print("IDENTIFYING BILL OUTCOMES BY CONGRESS")
    print("="*60)
    
    # Congress end dates
    congress_end_dates = {
        113: datetime(2015, 1, 3),
        114: datetime(2017, 1, 3),
        115: datetime(2019, 1, 3),
        116: datetime(2021, 1, 3),
        117: datetime(2023, 1, 3),
        118: datetime(2025, 1, 3)
    }
    
    current_date = datetime.now()
    
    # Process each congress
    for congress, end_date in congress_end_dates.items():
        congress_bills = df['congress'] == congress
        
        if current_date > end_date:
            # Congress has ended - all pending bills are failed
            pending = congress_bills & (df['passed'] == -1)
            df.loc[pending, 'passed'] = 0
            
            n_updated = pending.sum()
            if n_updated > 0:
                print(f"Congress {congress}: Marked {n_updated} pending bills as failed (congress ended)")
    
    # Categorize failure reasons
    failed_bills = df['passed'] == 0
    
    # Bills that never left committee
    never_left_committee = failed_bills & (
        (df['action_count'] <= 2) |
        (df['latest_action'].str.contains('referred to', case=False, na=False))
    )
    df.loc[never_left_committee, 'failure_reason'] = 'died_in_committee'
    
    # Bills with some progress
    some_progress = failed_bills & (
        (df['action_count'] > 2) & 
        (df['action_count'] <= 10) &
        (~never_left_committee)
    )
    df.loc[some_progress, 'failure_reason'] = 'stalled_in_process'
    
    # Bills that got far but didn't make it
    significant_progress = failed_bills & (
        (df['action_count'] > 10) |
        (df['latest_action'].str.contains('passed|reported', case=False, na=False))
    ) & (~some_progress) & (~never_left_committee)
    df.loc[significant_progress, 'failure_reason'] = 'failed_to_complete'
    
    # Any remaining failed bills
    remaining_failed = failed_bills & (df['failure_reason'].isna())
    df.loc[remaining_failed, 'failure_reason'] = 'congress_ended'
    
    # Final outcome distribution
    print(f"\nFinal outcome distribution across all congresses:")
    print(f"Passed: {(df['passed'] == 1).sum()} ({(df['passed'] == 1).mean()*100:.1f}%)")
    print(f"Failed: {(df['passed'] == 0).sum()} ({(df['passed'] == 0).mean()*100:.1f}%)")
    print(f"Pending: {(df['passed'] == -1).sum()}")
    
    # Outcome by congress
    print("\nOutcome rates by congress:")
    outcome_by_congress = pd.crosstab(df['congress'], df['passed'], normalize='index') * 100
    print(outcome_by_congress.round(1))
    
    return df

def add_temporal_features(df):
    """
    Add additional temporal features that don't rely on current date
    """
    
    print("\n" + "="*60)
    print("ADDING TEMPORAL FEATURES")
    print("="*60)
    
    # Month and quarter of introduction
    df['month_introduced'] = pd.to_datetime(df['introduced_date']).dt.month
    df['quarter_introduced'] = pd.to_datetime(df['introduced_date']).dt.quarter
    
    # Is it an election year?
    df['year_introduced'] = pd.to_datetime(df['introduced_date']).dt.year
    df['is_election_year'] = df['year_introduced'] % 2 == 0
    
    # Congressional session (1st or 2nd year)
    df['is_first_session'] = df['year_introduced'] % 2 == 1
    
    # Activity patterns
    df['early_activity'] = df['action_count'] / (df['days_active'].clip(upper=30) + 1)
    df['sustained_activity'] = df['action_count'] / (df['days_active'].clip(upper=180) + 1)
    
    # Committee engagement speed
    df['committee_engagement_speed'] = df['committee_count'] / (df['days_active'] / 30).clip(lower=1)
    
    # Drop year_introduced as it's too specific
    df = df.drop('year_introduced', axis=1)
    
    print("Added temporal features:")
    print("- month_introduced")
    print("- quarter_introduced") 
    print("- is_election_year")
    print("- is_first_session")
    print("- early_activity")
    print("- sustained_activity")
    print("- committee_engagement_speed")
    
    return df

def create_training_datasets(df):
    """
    Create training datasets for different purposes
    """
    
    print("\n" + "="*60)
    print("CREATING TRAINING DATASETS")
    print("="*60)
    
    # Full dataset with all features
    df.to_csv('../data/bills_6congress_full_enhanced.csv', index=False)
    print(f"Saved full enhanced dataset: {len(df)} bills")
    
    # Training dataset (only bills with known outcomes)
    training_df = df[df['passed'] != -1].copy()
    training_df.to_csv('../data/bills_6congress_training.csv', index=False)
    print(f"Saved training dataset: {len(training_df)} bills with known outcomes")
    
    return training_df

def analyze_dataset_characteristics(df):
    """
    Analyze the characteristics of the multi-congress dataset
    """
    
    print("\n" + "="*60)
    print("DATASET CHARACTERISTICS")
    print("="*60)
    
    # Pass rates by congress
    print("\nPass rates by congress:")
    for congress in sorted(df['congress'].unique()):
        congress_df = df[df['congress'] == congress]
        pass_rate = (congress_df['passed'] == 1).mean() * 100
        total_bills = len(congress_df)
        print(f"- {congress}th Congress: {pass_rate:.1f}% ({total_bills} bills)")
    
    # Overall statistics
    print(f"\nOverall statistics:")
    print(f"- Total bills: {len(df)}")
    print(f"- Overall pass rate: {(df['passed'] == 1).mean() * 100:.1f}%")
    print(f"- Bills with known outcomes: {(df['passed'] != -1).sum()}")
    
    # Feature availability
    print("\nFeature completeness:")
    for col in ['action_count', 'cosponsor_count', 'committee_count', 'is_bipartisan']:
        if col in df.columns:
            missing = df[col].isna().sum()
            print(f"- {col}: {(1 - missing/len(df)) * 100:.1f}% complete")
    
    # Bipartisan analysis
    if 'is_bipartisan' in df.columns:
        print(f"\nBipartisan bill analysis:")
        bipartisan_pass_rate = df[df['is_bipartisan'] == 1]['passed'].mean() * 100
        partisan_pass_rate = df[df['is_bipartisan'] == 0]['passed'].mean() * 100
        print(f"- Bipartisan bills pass rate: {bipartisan_pass_rate:.1f}%")
        print(f"- Partisan bills pass rate: {partisan_pass_rate:.1f}%")

def validate_data_quality(df):
    """
    Validate data quality and fix common issues
    """
    
    print("\n" + "="*60)
    print("VALIDATING DATA QUALITY")
    print("="*60)
    
    # Check for duplicate bills
    duplicates = df.duplicated(subset=['bill_id'])
    if duplicates.sum() > 0:
        print(f"⚠️ Found {duplicates.sum()} duplicate bills - removing...")
        df = df[~duplicates]
    
    # Check for missing critical fields
    critical_fields = ['bill_id', 'congress', 'bill_type', 'introduced_date', 'passed']
    for field in critical_fields:
        if field in df.columns:
            missing = df[field].isna().sum()
            if missing > 0:
                print(f"⚠️ Missing {field}: {missing} rows")
    
    # Ensure numeric fields are numeric
    numeric_fields = ['action_count', 'cosponsor_count', 'committee_count', 
                     'days_active', 'legislative_velocity']
    for field in numeric_fields:
        if field in df.columns:
            df[field] = pd.to_numeric(df[field], errors='coerce').fillna(0)
    
    # Ensure boolean fields are boolean
    boolean_fields = ['is_bipartisan', 'is_election_year', 'is_first_session']
    for field in boolean_fields:
        if field in df.columns:
            df[field] = df[field].astype(bool).astype(int)
    
    print("✅ Data validation complete")
    
    return df

def main():
    """
    Main preprocessing function
    """
    
    # Load the 6-congress dataset
    df = preprocess_six_congress_data()
    
    # Calculate smart days active (using latest action date)
    df = calculate_smart_days_active(df)
    
    # Identify outcomes considering ended congresses
    df = identify_outcomes_by_congress(df)
    
    # Add temporal features
    df = add_temporal_features(df)
    
    # Validate data quality
    df = validate_data_quality(df)
    
    # Create training datasets
    training_df = create_training_datasets(df)
    
    # Analyze dataset characteristics
    analyze_dataset_characteristics(training_df)
    
    print("\n" + "="*60)
    print("✅ PREPROCESSING COMPLETE!")
    print("="*60)
    print("\nCreated files:")
    print("- bills_6congress_full_enhanced.csv (all bills with features)")
    print("- bills_6congress_training.csv (bills with known outcomes)")
    print("\nKey improvements:")
    print("- Uses days to latest action instead of days since introduction")
    print("- Added legislative velocity feature")
    print("- Properly handles ended congresses")
    print("- Includes temporal features that don't leak future information")

if __name__ == "__main__":
    main()

PREPROCESSING 6-CONGRESS DATASET (113th-118th)
Loaded 76897 bills from 6 congresses

Congresses in dataset:
congress
113     9091
114    10233
115    11421
116    14345
117    15242
118    16565
Name: count, dtype: int64

CALCULATING SMART DAYS ACTIVE
Days active statistics:
- Mean: 49.0 days
- Median: 1.0 days
- Min: 1
- Max: 730

IDENTIFYING BILL OUTCOMES BY CONGRESS
Congress 113: Marked 8705 pending bills as failed (congress ended)
Congress 114: Marked 9735 pending bills as failed (congress ended)
Congress 115: Marked 10828 pending bills as failed (congress ended)
Congress 116: Marked 13979 pending bills as failed (congress ended)
Congress 117: Marked 14856 pending bills as failed (congress ended)
Congress 118: Marked 16258 pending bills as failed (congress ended)

Final outcome distribution across all congresses:
Passed: 2056 (2.7%)
Failed: 74841 (97.3%)
Pending: 0

Outcome rates by congress:
passed       0    1
congress           
113       96.8  3.2
114       96.7  3.3
115       